In [173]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


In [174]:
random_state=0
num_epochs = 1000
input_layer_size = 2
hidden_layer_1_size = 20
hidden_layer_2_size = 10
output_layer_size = 3

In [175]:
df_nls = pd.read_csv('Group20/Classification/NLS_Group20.txt', header=None, delimiter=" ", skiprows=1)
df_nls = df_nls.iloc[: , :-1]
df_nls = df_nls.rename(columns={0: 'a', 1: 'b'})

temp_df = [df_nls.iloc[:500,[0,1]], df_nls.iloc[500:1000,[0,1]], df_nls.iloc[1000:1500,[0,1]]]

df_train_data = pd.DataFrame(columns = ["a", "b", "class"])
df_valid_data = pd.DataFrame(columns = ["a", "b", "class"])
df_test_data = pd.DataFrame(columns = ["a", "b", "class"])

for i in range(3):
    temp_df[i]['class'] = i+1
    # split the dataset
    validDataPercentage = 20
    testDataPercentage = 20
    training_data, testing_data = train_test_split(temp_df[i], test_size=(testDataPercentage+validDataPercentage)*0.01, random_state=random_state)
    valid_data, testing_data = train_test_split(testing_data, test_size=(testDataPercentage/(testDataPercentage+validDataPercentage)), random_state=random_state)
    df_train_data = pd.merge(df_train_data, training_data, how='outer')
    df_valid_data = pd.merge(df_valid_data, valid_data, how='outer')
    df_test_data  = pd.merge(df_test_data , testing_data, how='outer')


df_train_data = df_train_data.sample(frac=1, random_state=random_state).reset_index(drop=True)
df_valid_data = df_valid_data.sample(frac=1, random_state=random_state).reset_index(drop=True)
df_test_data = df_test_data.sample(frac=1, random_state=random_state).reset_index(drop=True)

df_train_list = [df_train_data.loc[df_train_data['class'] == 1], df_train_data.loc[df_train_data['class'] == 2], df_train_data.loc[df_train_data['class'] == 3]]
df_valid_list = [df_valid_data.loc[df_valid_data['class'] == 1], df_valid_data.loc[df_valid_data['class'] == 2], df_valid_data.loc[df_valid_data['class'] == 3]]
df_test_list = [df_test_data.loc[df_test_data['class'] == 1], df_test_data.loc[df_test_data['class'] == 2], df_test_data.loc[df_test_data['class'] == 3]]


print(df_train_data.shape)
print(df_valid_data.shape)
print(df_test_data.shape)

(900, 3)
(300, 3)
(300, 3)


In [176]:
onehot_encoder = OneHotEncoder(sparse=False)
y_onehot = onehot_encoder.fit_transform(df_train_data.iloc[:,-1].to_numpy().reshape(-1, 1))
y_onehot

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [177]:
class FCNN:
    def __init__(self, input_layer_size, hidden_layer_1_size, hidden_layer_2_size, output_layer_size):
        # Input to hidden layer1 Matrix: (d+1) x j
        np.random.seed(random_state)
        self.wij = np.random.randn(input_layer_size, hidden_layer_1_size)
        self.b1 = np.zeros((1, hidden_layer_1_size))
        
        # hidden layer1 to hidden layer2 Matrix: (j+1) x l
        self.wjl = np.random.randn(hidden_layer_1_size, hidden_layer_2_size)
        self.b2 = np.zeros((1, hidden_layer_2_size))
        
        # Output Matrix: (l+1) x k
        self.learning_rate = 1
        self.wlk = np.random.randn(hidden_layer_2_size, output_layer_size)
        self.b3 = np.zeros((1, output_layer_size))

    def forward(self, X):
        self.a1 = np.dot(X, self.wij) + self.b1
        # self.a1 = np.dot(X, self.wij)
        self.h1 = self.g(self.a1)

        self.a2 = np.dot(self.h1, self.wjl) + self.b2
        # self.a2 = np.dot(self.h1, self.wjl)
        self.h2 = self.g(self.a2)

        self.a3 = np.dot(self.h2, self.wlk) + self.b3
        # self.a3 = np.dot(self.h2, self.wlk)
        self.h3 = self.f(self.a3)
        
        return self.h3
    
    def g(self, aj):
        return np.tanh(aj)
        # return 1/(1+np.exp(-aj))

    def f(self, ak):
        return 1/(1+np.exp(-ak))

    def softmax(self, x):
        exp_x = np.exp(x)
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def backward(self, X, y, y_pred):
        m = X.shape[0]
        dz3 = y_pred - y
        dW3 = np.dot(self.h2.T, dz3) / m
        db3 = np.sum(dz3, axis=0, keepdims=True) / m
        dz2 = np.dot(dz3, self.wlk.T) * (1 - np.power(self.h2, 2))
        dW2 = np.dot(self.h1.T, dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
        dz1 = np.dot(dz2, self.wjl.T) * (1 - np.power(self.h1, 2))
        dW1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m

        self.wlk -= self.learning_rate * dW3
        self.b3 -= self.learning_rate * db3
        self.wjl -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2
        self.wij -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1

    def train(self, X, y):
        X = X.iloc[:,:-1]
        # X.insert(0,'hat',[1]*X.shape[0])
        # print(X)
        losses = []
        for i in range(num_epochs):
            self.learning_rate = 1/(i+1)
            y_pred = self.forward(X)
            loss = self.cross_entropy_loss(y_pred, y)
            losses.append(loss)
            self.backward(X, y_onehot, y_pred)
            # if i % 100 == 0:
                # print(f"Epoch: {i}, Loss: {loss}")
        return losses
    
    def predict(self, X):
        y_pred = self.forward(X)
        return np.argmax(y_pred, axis=1)

    def cross_entropy_loss(self, y_pred, y_true):
        m = y_true.shape[0]
        log_likelihood = -np.log(y_pred[range(m), y_true.argmax(axis=1)])
        loss = np.sum(log_likelihood) / m
        return loss


In [178]:
fcnn = FCNN(input_layer_size, hidden_layer_1_size, hidden_layer_2_size, output_layer_size)
losses = fcnn.train(df_train_data, y_onehot)

y_pred = np.argmax(fcnn.forward(df_test_data.iloc[:,:-1]), axis=1)
y_pred = np.argmax(fcnn.forward(df_test_data.iloc[:,:-1]), axis=1) + 1
accuracy = np.mean(y_pred == df_test_data.iloc[:,-1])
print(f'Accuracy: {accuracy:.2f}')



Accuracy: 0.93


In [179]:
import pandas as pd

# create a sample dataframe
df = pd.DataFrame({0:[1, 2, 3],1:[4, 5, 6]})

# insert a new column at the beginning
df.insert(0,'hat',[1]*df.shape[0])

print(df)


   hat  0  1
0    1  1  4
1    1  2  5
2    1  3  6
